In [1]:
import xarray as xr
from tqdm import tqdm
import numpy as np

# -------------------------------
# CONFIGURATION
# -------------------------------
ZARR_PATH = "gs://weatherbench2/datasets/era5/1959-2023_01_10-wb13-6h-1440x721_with_derived_variables.zarr"
NETCDF_OUTPUT = "era5_europe_2021Q1.nc"

TIME_SLICE = slice("1959", "2023")
LAT_BOUNDS = slice(75, 30)  # Europe
LON_BOUNDS = lambda lon: (lon >= 335) | (lon <= 50)

VARIABLES = [
    'total_precipitation_6hr',
    '2m_temperature', '2m_dewpoint_temperature', 'surface_pressure',
    'mean_sea_level_pressure', '10m_u_component_of_wind', '10m_v_component_of_wind',
    '10m_wind_speed', 'u_component_of_wind', 'v_component_of_wind',
    'total_column_water_vapour', 'integrated_vapor_transport', 'boundary_layer_height',
    'specific_humidity', 'total_cloud_cover',
    'mean_surface_net_short_wave_radiation_flux',
    'mean_surface_latent_heat_flux', 'mean_surface_sensible_heat_flux',
    'snow_depth', 'sea_surface_temperature', 'volumetric_soil_water_layer_1',
    'mean_vertically_integrated_moisture_divergence', 'eddy_kinetic_energy',
    'land_sea_mask'
]

# -------------------------------
# STEP 1: Load and subset ZARR
# -------------------------------
print("🔹 Opening ERA5 Zarr dataset...")
ds = xr.open_zarr(ZARR_PATH, consolidated=True, storage_options={"token": "anon", "asynchronous": False})

print("🔹 Subsetting region and time...")
ds = ds.sel(time=TIME_SLICE, latitude=LAT_BOUNDS)
ds = ds.where(LON_BOUNDS(ds.longitude), drop=True)
ds = ds[VARIABLES]

# -------------------------------
# STEP 2: Trigger computation with progress
# -------------------------------
print("🔹 Triggering compute and showing progress...")
# Wrap all variables in tqdm
with tqdm(total=len(VARIABLES), desc="Computing Variables") as pbar:
    computed_vars = {}
    for var in VARIABLES:
        computed_vars[var] = ds[var].load()
        pbar.update(1)

ds_computed = xr.Dataset(computed_vars)

# -------------------------------
# STEP 3: Save to NetCDF
# -------------------------------
print(f"💾 Saving NetCDF to: {NETCDF_OUTPUT} ...")
ds_computed.to_netcdf(NETCDF_OUTPUT)
print("✅ Done. File saved.")


🔹 Opening ERA5 Zarr dataset...


c:\Users\peera\.conda\envs\drought_lstm_base\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.1 is exactly one major version older than the runtime version 6.31.1 at api.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\peera\.conda\envs\drought_lstm_base\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🔹 Subsetting region and time...
🔹 Triggering compute and showing progress...


Computing Variables:   0%|          | 0/24 [02:41<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import xarray as xr
import numpy as np

class GriddedClimateDataset(Dataset):
    def __init__(self, file_path, input_len=12, forecast_len=1, variables=None,
                 target_variable=None, lat_bounds=(75, 30), lon_bounds=(335, 50),
                 time_slice=slice("1959-01", "2023-01")):
        
        if file_path.endswith(".nc"):
            self.ds = xr.open_dataset(file_path)
        else:
            self.ds = xr.open_zarr(file_path, consolidated=True, decode_times=False,
                                   storage_options={"token": "anon", "asynchronous": False})
            from cftime import num2date
            units = self.ds.time.attrs.get("units", "hours since 1900-01-01 00:00:0.0")
            cal = self.ds.time.attrs.get("calendar", "standard")
            self.ds["time"] = ("time", num2date(self.ds.time.values, units, calendar=cal))

        # Subset region and time
        self.ds = self.ds.sel(time=time_slice, latitude=slice(*lat_bounds))
        self.ds = self.ds.where((self.ds.longitude >= lon_bounds[0]) | 
                                (self.ds.longitude <= lon_bounds[1]), drop=True)

        # Process each variable to ensure 3D (time, lat, lon)
        processed_vars = []
        for var in variables:
            da = self.ds[var]
            if "level" in da.dims:
                da = da.mean(dim="level")  # reduce level
            if set(da.dims) == {"latitude", "longitude"}:  # static 2D
                da = da.expand_dims(time=self.ds.time)
            processed_vars.append(da)

        # Stack into [time, variable, lat, lon]
        stacked = xr.concat(processed_vars, dim="variable")
        stacked = stacked.transpose("time", "variable", "latitude", "longitude")

        self.data = stacked
        self.target = self.ds[target_variable].transpose("time", "latitude", "longitude")
        self.time_coords = self.ds.time.values
        self.input_len = input_len
        self.forecast_len = forecast_len
        self.variables = variables
        self.target_variable = target_variable
        self.length = len(self.data.time) - input_len - forecast_len + 1



# --- Example Training Loop for ConvLSTM / MESA-NET Compatible ---
def train_one_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for x, y in dataloader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        output = model(x)  # Expected output shape: [B, forecast_len, H, W]
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(dataloader)

# Usage
INPUT_LEN = 12  # Number of input time steps
FORECAST_LEN = 4  # Number of forecast time steps
PATH = "gs://weatherbench2/datasets/era5/1959-2023_01_10-wb13-6h-1440x721_with_derived_variables.zarr"
VARIABLES = [
    'total_precipitation_6hr',
    '2m_temperature', '2m_dewpoint_temperature', 'surface_pressure',
    'mean_sea_level_pressure', '10m_u_component_of_wind', '10m_v_component_of_wind',
    '10m_wind_speed', 'u_component_of_wind', 'v_component_of_wind',
    'total_column_water_vapour', 'integrated_vapor_transport', 'boundary_layer_height',
    'specific_humidity', 'total_cloud_cover',
    'mean_surface_net_short_wave_radiation_flux',
    'mean_surface_latent_heat_flux', 'mean_surface_sensible_heat_flux',
    'snow_depth', 'sea_surface_temperature', 'volumetric_soil_water_layer_1',
    'mean_vertically_integrated_moisture_divergence', 'eddy_kinetic_energy',
    'land_sea_mask'
]
TARGET_VARIABLE = 'total_precipitation_6hr'
dataset = GriddedClimateDataset(PATH, input_len=INPUT_LEN, forecast_len=FORECAST_LEN, 
                                  variables=VARIABLES, target_variable=TARGET_VARIABLE)
# dataloader = DataLoader(dataset, batch_size=8, shuffle=True)
# model = YourConvLSTMModel(...).to(device)
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
# criterion = torch.nn.MSELoss()
print("Data shape:", dataset.data.shape)
print("Target shape:", dataset.target.shape)

Data shape: (93544, 24, 181, 301)
Target shape: (93544, 181, 301)


In [12]:
x, y = dataset[0]
print("Sample x shape:", x.shape)  # should be [input_len, num_vars, H, W]
print("Sample y shape:", y.shape)  # should be [forecast_len, H, W] → [6, 181, 301]

NotImplementedError: Subclasses of Dataset should implement __getitem__.